# Stats analysis

This notebook performins a one-way analysis of variance (ANOVA) or a Kruskal-Wallis test on the classification accuracies of the `WANG2016` dataset. 
- If the classification data if found to have independence of observations, be normally distributed, and have homogeneity of variances, the one-way ANOVA was performed. 
- Otherwise, the Kruskal-Wallis test will take place. 

The interest of doing the one-way ANOVA is to test the null hypothesis ($\text{H}_0$) that one of the groups is different from the other. Our expectation is that we will reject the null hypothesis. Thus, implying that the proposed classifiers of the SSVEP toolbox perform as well as the Riemmanian geometry classifier suggested by the MOABB tool. However, if the null hypothesis is not rejected, then a post-hoc test with a Bonferroni correction for multiple comparisons were used to compare the classifiers.

## Import libraries

In [23]:
import numpy as np
import pandas as pd
from scipy.stats import shapiro, f_oneway, kruskal
from statsmodels.stats.multitest import multipletests
from pingouin import sphericity
import matplotlib.pyplot as plt
import scikit_posthocs as sp

## Import data

In [7]:
results_file = "results_ncan_moabb.csv"

data = pd.read_csv(results_file)
data.set_index('Subject', inplace=True)

# Normality and variance test

Do a Shapiro-Wilk test for normality and a Mauchly's test for sphericity to check the variance of the data

In [11]:
def check_normality(data):
    stat, pvalue = shapiro(data)
    return pvalue >= 0.05

# Mauchly's test for sphericity
def check_sphericity(data):
    result = sphericity(
        data,
        method='mauchly',
        alpha=0.05
        )
    return result

# Check normality and sphericity for each column
normality_passed = all(check_normality(data[col]) for col in data.columns)
sphericity_passed = check_sphericity(data).spher

# Statistical tests

If normality and sphericity test have passed, perform a one-way ANOVA. Otherwise, perform a Kruskal-Wallis test

In [15]:
if (normality_passed and sphericity_passed):
    print("Data is normal and spheric")
    stat, pvalue = f_oneway(*[data[col] for col in data.columns])
    print(f"- One-way ANOVA p-value: {pvalue}")
else:
    print("Data is not normal or not spheric")
    stat, pvalue = kruskal(*[data[col] for col in data.columns])
    print(f"- Kruskal-Wallis p-value: {pvalue}")

Data is not normal or not spheric
- Kruskal-Wallis p-value: 2.2164269995166644e-07


## Multiple test comparison

Do a Bonferroni correction to do a multiple test comparison

In [20]:
from scipy.stats import f_oneway, friedmanchisquare
stats = []
pvalues = []
if normality_passed and sphericity_passed:
    # Perform repeated measures ANOVA
    stats.append, pvalue.append = f_oneway(*[data[col] for col in data.columns])
    print(f"Repeated Measures ANOVA p-value: {pvalue}")
else:
    # Perform Friedman's test
    stats.append(friedmanchisquare(*[data[col] for col in data.columns])[0])
    pvalues.append(friedmanchisquare(*[data[col] for col in data.columns])[1])
    print(f"Friedman's test p-value: {pvalue}")


Friedman's test p-value: 5.58706842353853e-12


In [21]:
alpha = 0.05

# Get the column names from the data
columns = data.columns

# Perform Friedman's test for all possible combinations of comparing the columns
pvalues = []
for i in range(len(columns)):
    for j in range(i+1, len(columns)):
        stat, pvalue = friedmanchisquare(data[columns[i]], data[columns[j]])
        pvalues.append(pvalue)

# Apply Bonferroni correction to the p-values
alpha_corrected = alpha / len(pvalues)
rejected, p_adjusted, _, _ = multipletests(pvalues, alpha=alpha_corrected, method='bonferroni')

# Count the number of significant comparisons
num_significant = sum(rejected)

# Print the results
print(f"Number of significant comparisons: {num_significant}")
print("Adjusted p-values:")
for i in range(len(p_adjusted)):
    print(f"{columns[i]} vs {columns[i+1]}: {p_adjusted[i]}")

ValueError: At least 3 sets of samples must be given for Friedman test, got 2.

https://scikit-posthocs.readthedocs.io/en/latest/generated/scikit_posthocs.posthoc_nemenyi_friedman.html


In [24]:
result = sp.posthoc_nemenyi_friedman(data)
print(result)

              fbCCA       MSI    MEC  RG_logreg
fbCCA      1.000000  0.180649  0.001      0.001
MSI        0.180649  1.000000  0.001      0.001
MEC        0.001000  0.001000  1.000      0.900
RG_logreg  0.001000  0.001000  0.900      1.000
